In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121
from keras.models import Model
from keras.layers import Dense, Flatten, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

#Đường dẫn thư mục train và test
train_path = r'/content/drive/MyDrive/Colab Notebooks/Project X-ray/train'
test_path = r'/content/drive/MyDrive/Colab Notebooks/Project X-ray/test'
val_path = r'/content/drive/MyDrive/Colab Notebooks/Project X-ray/val'

batch_size = 16

img_height = 500
img_width = 500

ModuleNotFoundError: No module named 'google.colab'

In [2]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2
)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = image_gen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

test = test_data_gen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    color_mode='rgb',
    shuffle=False,
    class_mode='binary',
    batch_size=batch_size
)

validation_generator = image_gen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

Found 1943 images belonging to 2 classes.
Found 611 images belonging to 2 classes.
Found 497 images belonging to 2 classes.


In [4]:
base_model = DenseNet121(include_top=False, input_shape=(img_height, img_width, 3))
for layer in base_model.layers[:-4]:  # Fine-tune the last 4 layers
    layer.trainable = False

# Add custom layers on top of the pre-trained model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
cnn = Model(inputs=base_model.input, outputs=predictions)
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.3, min_lr=0.000001)
callbacks_list = [early, learning_rate_reduction]

# Compute class weights
from sklearn.utils.class_weight import compute_sample_weight

unique_classes = np.unique(train_generator.classes)
weights = compute_sample_weight(class_weight='balanced', y=train_generator.classes)
cw = dict(zip(unique_classes, weights))
print(cw)

# Train the model
cnn.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    class_weight=cw,
    callbacks=callbacks_list
)

29084464/29084464 [==============================] - 2s 0us/step
{0: 0.6979166666666666, 1: 0.6979166666666666}
Epoch 1/25
122/122 [==============================] - 1129s 9s/step - loss: 0.4531 - accuracy: 0.6475 - val_loss: 0.5791 - val_accuracy: 0.7123 - lr: 0.0010
Epoch 2/25
122/122 [==============================] - 219s 2s/step - loss: 0.3635 - accuracy: 0.7422 - val_loss: 0.6616 - val_accuracy: 0.7304 - lr: 0.0010
Epoch 3/25
122/122 [==============================] - 256s 2s/step - loss: 0.3424 - accuracy: 0.7612 - val_loss: 0.5046 - val_accuracy: 0.7706 - lr: 0.0010
Epoch 4/25
122/122 [==============================] - 215s 2s/step - loss: 0.3425 - accuracy: 0.7627 - val_loss: 0.4589 - val_accuracy: 0.7807 - lr: 0.0010
Epoch 5/25
122/122 [==============================] - 213s 2s/step - loss: 0.3226 - accuracy: 0.7751 - val_loss: 0.9376 - val_accuracy: 0.7143 - lr: 0.0010
Epoch 6/25
122/122 [==============================] - 213s 2s/step - loss: 0.3185 - accuracy: 0.7792 - val_

In [8]:
cnn.save('DenseNet_custom.h5')
from google.colab import files
files.download('DenseNet_custom.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
from sklearn.metrics import confusion_matrix, classification_report

# Evaluate the model on the test data
test_accu = cnn.evaluate(test)
print('The testing accuracy is:', test_accu[1] * 100, '%')

# Predict the labels for the test data
y_pred = cnn.predict(test)
y_pred = np.round(y_pred).astype(int)  # Chuyển đổi dự đoán thành nhãn nhị phân (0 hoặc 1)

# Get the true labels
y_true = test.classes

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Compute classification report
class_report = classification_report(y_true, y_pred, target_names=['Normal', 'Disease'])
print('Classification Report:')
print(class_report)

# Extract Sensitivity, Specificity, and F1-Score from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)
recall = sensitivity  # Recall là tên khác của Sensitivity
f1_score = 2 * (precision * recall) / (precision + recall)

print(f'Sensitivity: {sensitivity * 100:.2f}%')
print(f'Specificity: {specificity * 100:.2f}%')
print(f'F1-Score: {f1_score:.2f}')

39/39 [==============================] - 512s 13s/step - loss: 0.4473 - accuracy: 0.7840
The testing accuracy is: 78.39607000350952 %
39/39 [==============================] - 24s 563ms/step
Confusion Matrix:
[[401  13]
 [119  78]]
Classification Report:
              precision    recall  f1-score   support

      Normal       0.77      0.97      0.86       414
     Disease       0.86      0.40      0.54       197

    accuracy                           0.78       611
   macro avg       0.81      0.68      0.70       611
weighted avg       0.80      0.78      0.76       611

Sensitivity: 39.59%
Specificity: 96.86%
F1-Score: 0.54
